In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'

x = torch.randn(100, 1, dtype=torch.float)
y = 1 + 2 * x + 0.1 * torch.randn(100, 1, dtype=torch.float)

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MyDataSet(Dataset):
  def __init__(self, x_tensor, y_tensor):
    self.x = x_tensor
    self.y = y_tensor
  def __getitem__(self, index):
    return (self.x[index], self.y[index])
  def __len__(self):
    return len(self.x)

dataset = MyDataSet(x, y)

train_dataset, val_dataset = random_split(dataset, [80, 20])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=20, shuffle=True)


In [3]:
lr = 1e-1
epochs = 1000

model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

val_losses = []

def get_train_step(model, loss_fn, optimizer):
  def train_step(x, y):
    yhat = model(x)
    loss = loss_fn(yhat, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()
  return train_step

train_step = get_train_step(model, loss_fn, optimizer)

for epoch in range(epochs):
  for x_batch, y_batch in train_dataloader:
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    train_step(x_batch, y_batch)
  with torch.no_grad():
    for x_batch, y_batch in val_dataloader:

      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)

      model.eval()
      yhat = model(x_batch)
      val_loss = loss_fn(yhat, y_batch)
      val_losses.append(val_loss.item())


print(model.state_dict())
# print(val_losses)
# plt.plot(val_losses[4:])

              

OrderedDict([('0.weight', tensor([[1.9966]])), ('0.bias', tensor([0.9995]))])
